# current issue is in finding reactions where species have '(' or ')'

This notebook describes some of the methods in this package and how they can be used.

In [2]:
import cantera_tools as ctt
import numpy as np
from scipy import integrate
import cantera as ct
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
import importlib
importlib.reload(ctt)

<module 'cantera_tools' from '/home/mark/workspace/tools/cantera_tools.py'>

## running a simple simulation

In [6]:
model_link = 'cookbook_files/model.cti'

# creates the cantera Solution object
solution = ctt.create_mechanism(model_link)

# finds initial mole fraction for a fuel-air ratio of 1 with 30%/70% methanol/propane blend
mole_fractions = ctt.get_initial_mole_fractions(stoich_ratio = 1,
                                               fuel_mole_ratios = [.3,.7],
                                               oxygen_per_fuel_at_stoich_list = [1.5,5],
                                               fuels = ['CH3OH','C3H8'])

# set initial conditions of solution in kelvin pascals and mole fractions
solution.TPX = 750, 10**6, mole_fractions

# setup adiabatic constant volume reactor
reactor = ct.IdealGasReactor(solution)
simulator = ct.ReactorNet([reactor])
solution = reactor.kinetics

# setup data storage using functions from this class - this saves states, species and reaction profiles
df = pd.DataFrame()
df = ctt.append_data_to_df(simulator, solution, df, add_rxns=True)

# run simulation
time_final = 50 #seconds
while simulator.time < time_final:
    simulator.step(time_final)
    # appends data to DataFrame
    df = ctt.append_data_to_df(simulator,solution,df, add_rxns=True)


# data can be saved to avoid rerunning the simulation for data analysis (in most cases)
df.to_pickle('cookbook_files/%s.pic' % (mole_fractions))

## reducing a mechanism by reactions
The modules can create a reduced mechanism given a list of desired reaction strings, using how cantera represents the reaction strings (this can be found by `solution.reaction_equations()`). It will remove any unused species as well.

In [23]:
desired_reactions = ['CH3OH + O2 <=> CH2OH(29) + HO2(12)',
 'C3H8 + O2 <=> C3H7(61) + HO2(12)',
 'C3H8 + O2 <=> C3H7(60) + HO2(12)',
 'CH3OH + OH(10) <=> CH2OH(29) + H2O(11)',
 'C3H8 + OH(10) <=> C3H7(60) + H2O(11)',
 'C3H8 + OH(10) <=> C3H7(61) + H2O(11)',
 'CH3OH + HO2(12) <=> CH2OH(29) + H2O2(13)',
 'C3H8 + HO2(12) <=> C3H7(61) + H2O2(13)',
 'C3H8 + HO2(12) <=> C3H7(60) + H2O2(13)',
 'C3H7(60) + O2 <=> C3H7O2(78)',
 'C3H7(61) + O2 <=> C3H7O2(80)',]

# make the reduced mechanism using the full mechanism `.cti` file.
solution_reduced = ctt.create_mechanism(model_link, kept_reaction_equations=desired_reactions)

# NOTE: this cantera Solution object can now be used like any other

## analyzing data
### obtaining reaction and species data

In [10]:
# this outputs a dataframe of just species
data_species = ctt.find_species(df)

# this outputs a dataframe of just reactions
data_reactions = ctt.find_reactions(df)

# this outputs a dataframe with just reactions involving methanol
reactions_with_methanol = ctt.find_reactions(df,species = 'CH3OH')

### finding stoichiometric coefficients

In [8]:
# this is wrapper function that outputs the stoichiometric coefficient given reaction strings:
ctt.obtain_stoichiometry_of_species(solution,'CH2OH(29)','CH3OH + HO2(12) <=> CH2OH(29) + H2O2(13)')

1.0

In [27]:
# it can also deal with a list of reactions
reactions_with_species = ctt.find_reactions(df,'CH2OH(29)')
ctt.obtain_stoichiometry_of_species(solution,'CH2OH(29)',reactions_with_species)

Exception: No reactions found for species CH2OH(29)

In [24]:
reactions_with_species

""
0
1
2
3
4
5
6
7
8
9


## analyzing data
### finding stoichiometric coefficients